In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
from taigapy import create_taiga_client_v3

tc = create_taiga_client_v3()

In [ ]:
prism_auc = tc.get('internal-24q4-8c04.117/PRISMOncologyReferenceAUCMatrix')

In [ ]:
prism_auc

In [ ]:
hs_mutations = tc.get('internal-24q4-8c04.117/OmicsSomaticMutationsMatrixHotspot')

In [ ]:
hs_mutations

In [ ]:
expr = tc.get('internal-24q4-8c04.117/OmicsExpressionProteinCodingGenesTPMLogp1BatchCorrected')

In [ ]:
portal_compounds = tc.get('internal-24q4-8c04.117/PortalCompounds')

In [ ]:
portal_compounds

In [ ]:
compounds_exploded = portal_compounds[['CompoundID','SampleIDs']
                                        ].set_index('CompoundID').SampleIDs.str.split(';').explode()

compounds_exploded

In [ ]:
comp_meta_expl = (
        portal_compounds.drop('SampleIDs', axis =1)
        .merge(compounds_exploded.rename('SampleID'), left_on='CompoundID', right_index=True)

)

comp_meta_expl

In [ ]:
prism_compounds = comp_meta_expl[comp_meta_expl.SampleID.str.startswith('BRD')]
prism_compounds['SampleID'] = prism_compounds.SampleID.str.extract(r'BRD:([A-Za-z0-9\-]+)')[0]
oncref_compounds = prism_compounds[prism_compounds.SampleID.str.startswith('PRC')]
oncref_compounds
                                   

In [ ]:
mdm_inhib = oncref_compounds[~oncref_compounds.GeneSymbolOfTargets.isna() &
                 oncref_compounds.GeneSymbolOfTargets.str.contains('MDM')].SampleID

mdm_inhib

In [ ]:
hs_mutations['TP53 (7157)']>0

In [ ]:
mdm_data = (
    prism_auc[mdm_inhib].melt(value_vars=mdm_inhib.to_list(),ignore_index=False).reset_index()
    .rename(columns={'index':'ModelID','variable':'PRC_ID','value':'AUC'})
    .dropna().merge(oncref_compounds[oncref_compounds.SampleID.isin(mdm_inhib)],
                   left_on = 'PRC_ID', right_on='SampleID')
    .merge((hs_mutations['TP53 (7157)']>0), left_on = 'ModelID', right_index = True)
    .merge(expr['MDM2 (4193)'],left_on = 'ModelID', right_index = True)
    .rename(columns={'TP53 (7157)': 'TP53_mut', 'MDM2 (4193)': 'MDM2_expr'})

)

mdm_data

In [ ]:
g = sns.FacetGrid(mdm_data,col='CompoundName', col_wrap=3, hue='TP53_mut')
g.map(sns.scatterplot,'MDM2_expr', 'AUC')

In [ ]:
sns.lmplot(data=mdm_data, col='CompoundName', hue='TP53_mut', x='MDM2_expr', y='AUC')

In [ ]:
hs_mutations

In [ ]:
gene = tc.get('internal-24q4-8c04.117/Gene')

In [ ]:
# Note here Int64 is a nullable integer which allows for NA and Inf values

gene['depmap'] = gene.symbol + ' (' + gene.entrez_id.astype('Int64').astype(str) + ')'

In [ ]:
gene[['ensembl_gene_id', 'depmap']].set_index('depmap')['ensembl_gene_id']

In [ ]:
hs_mutations.columns = (
    hs_mutations.columns.map(gene[['ensembl_gene_id', 'depmap']]
    .set_index('depmap')['ensembl_gene_id'])

)

In [ ]:
hs_mutations

In [ ]:
gene[['ensembl_gene_id', 'depmap']].set_index('depmap')['ensembl_gene_id']

# Coocccuring mutations

In [ ]:
hs_mutations = tc.get('internal-24q4-8c04.117/OmicsSomaticMutationsMatrixHotspot')

In [ ]:
hs_mutations.columns = hs_mutations.columns.str.extract((r'([A-Za-z0-9\-]+) '))[0]

In [ ]:
hs_mutations

In [ ]:
kras_egfr = hs_mutations[['KRAS', 'EGFR']]

In [ ]:
kras_egfr

In [ ]:
kras_egfr = kras_egfr > 0

In [ ]:
from scipy import stats

In [ ]:
c_tab = kras_egfr.value_counts().reset_index().pivot(index='KRAS',columns='EGFR').fillna(0)
c_tab

In [ ]:
ftest = stats.fisher_exact()

In [ ]:
hs_mutations = hs_mutations > 0

In [ ]:
hf_muts = hs_mutations.loc[:, hs_mutations.sum() > 4]

In [ ]:
def contingency_mat(gene_1, gene_2, muts):
    out = (muts[[gene_2, gene_1]].value_counts().reset_index().pivot(index = gene_1, columns = gene_2))
    out = out.fillna(0).astype(int)
    out.index = [gene_1 + '_WT', gene_1+'_MUT']
    out.columns = [gene_2 + '_WT', gene_2+'_MUT']
    return out
    

In [ ]:
contingency_mat('EGFR', 'KRAS', hf_muts)

In [ ]:
contingency_mat('EGFR', 'PTEN', hf_muts)

In [ ]:
def is_powered(gene_1, gene_2, mut):
    mat = contingency_mat(gene_1, gene_2, mut)
    tested = stats.fisher_exact(mat)
    out = pd.DataFrame(data = [[tested.statistic, tested.pvalue]],
                       columns = ['odds_ratio', 'pval'], index = [[gene_1], [gene_2]])
    return out

In [ ]:
is_powered('BRAF', 'PTEN', hf_muts)

In [ ]:
gene_pair_result = []

for gene_1, gene_2 in pd.MultiIndex.from_product([hf_muts.columns, hf_muts.columns]):
    if gene_1 == gene_2:
        continue
    gene_pair_result.append(is_powered(gene_1, gene_2, hf_muts))

df_cooc_results = pd.concat(gene_pair_result)
df_cooc_results['qval'] = stats.false_discovery_control(df_cooc_results.pval)

In [ ]:
df_cooc_results

In [ ]:
df_cooc_results['log_q'] = -df_cooc_results.qval.apply(np.log10)
df_cooc_results['log_odds'] = df_cooc_results.odds_ratio.apply(np.log2)
df_cooc_results['highlight'] = (df_cooc_results.odds_ratio > 300) | (df_cooc_results.qval < 0.2)

In [ ]:
sns.scatterplot(data=df_cooc_results, x = 'log_odds', y ='log_q', hue = 'highlight')

In [ ]:
px.scatter(df_cooc_results, x='log_odds', y='log_q', color='highlight')

# CRISPR Drug associations

In [ ]:
crispr = tc.get('internal-24q4-8c04.117/CRISPRGeneEffect')

In [ ]:
crispr

In [ ]:
crispr.head()

In [ ]:
crispr.isna().any(axis = 0).mean(), crispr.isna().any(axis = 1).mean()

In [ ]:
sns.heatmap(crispr.isna())

In [ ]:
prism_auc.isna().any(axis = 0).mean(), prism_auc.isna().any(axis = 1).mean()

In [ ]:
crispr.shape

In [ ]:
prism_auc.shape

In [ ]:
sns.heatmap(prism_auc.isna())

In [ ]:
sns.clustermap(prism_auc.isna())

In [ ]:
sns.clustermap(prism_auc.dropna(axis = 0, thresh = (len(prism_auc.columns)-5)).isna())

In [ ]:
prism_filtered = prism_auc.dropna(axis=0, thresh=len(prism_auc.columns) - 5)
prism_filtered = prism_filtered.dropna(axis=1)

In [ ]:
prism_filtered.isna().any(axis = 0).sum(), prism_filtered.isna().any(axis = 1).sum()

In [ ]:
crispr = crispr.dropna(axis=1)

In [ ]:
crispr.head()

In [ ]:
pd.concat([crispr, prism_filtered], axis = 1).dropna(axis=0).corr()

In [ ]:
pd.concat([crispr, prism_filtered], axis = 1)

In [ ]:
def subset_corr(mat, idxs_0, idxs_1):
    d = mat.to_numpy()
    sums = d.sum(0, keepdims=True)
    stds = d.std(0, keepdims=True)
    n = d.shape[0]

    k=len(idxs_0)
    l=len(idxs_0.union(idxs_1))
    d2 = d[:, :k].T.dot(d[:, k:l])
    sums2 = sums[:, :k].T.dot(sums[:, k:l])
    stds2 = sums[:, :k].T.dot(stds[:, k:l])

    out = pd.DataFrame((d2 - sums2 / n) / stds2 / n, mat.columns[:k], mat.columns[k:l])

    return out
                

In [ ]:
corr_output = subset_corr(pd.concat([crispr, prism_filtered], axis = 1).dropna(axis=0),
               crispr.columns, 
               prism_filtered.columns)

In [ ]:
corr_output

In [ ]:
# Rank each gene by how correlated it is with each drug

gene_ranks = corr_output.rank(axis=1, ascending=False)

drug_ranks = corr_output.rank(axis=0, ascending=False)


In [ ]:
oncref_compounds[(oncref_compounds.GeneSymbolOfTargets.str.contains('BRAF')) 
                 & oncref_compounds.SampleID.isin(prism_filtered.columns)]

In [ ]:
gene_ranks['PRC-004799305-255-84'].sort_values(ascending=True)